In [32]:
#Importing libraries
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas


**0. Data reading and cleaning**

In [33]:
#Reading the dataset
df = pd.read_csv('CO2_Emissions_1960-2018.csv')

#Defining the countries
country = df[df.columns[0]]

#Selecting the values from 2000 to 2018
years = df[df.columns[41:,]]

#Making a dataframe with emissions by country since 2000 to 2018
df_2 =  pd.concat([country, years], axis=1)
df_2.head()

,Country Name,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,0.900399,0.971981,0.975749,1.009963,1.057182,1.032998,1.023646,1.045968,1.086449,1.021954,1.048876,1.005338,1.021646,1.031833,1.041145,0.987393,0.971016,0.959978,0.933541
2,Afghanistan,0.037055,0.037488,0.048671,0.057008,0.045700,0.063927,0.073393,0.087083,0.158356,0.211306,0.297065,0.407074,0.335351,0.263716,0.234037,0.232176,0.208857,0.203328,0.200151
3,Africa Western and Central,0.447244,0.474140,0.474875,0.474592,0.474318,0.509458,0.487953,0.475148,0.475025,0.426770,0.472819,0.497023,0.490867,0.504655,0.507671,0.480743,0.472959,0.476438,0.515544
4,Angola,0.754476,0.761843,0.777423,0.993848,1.028354,0.937037,1.009434,1.123635,1.155993,1.205902,1.221515,1.216317,1.204799,1.261542,1.285365,1.260921,1.227703,1.034317,0.887380


In [34]:
#Checking the dataframe to look for countries with NaN Values
df_2.info()

#Countries without CO2 emissions information since 2000 to 2018
countries_nan_value = df_2[df_2.isna().any(axis=1)]
countries_no_info = countries_nan_value[countries_nan_value.columns[0]].tolist()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  266 non-null    object 
 1   2000          239 non-null    float64
 2   2001          239 non-null    float64
 3   2002          239 non-null    float64
 4   2003          239 non-null    float64
 5   2004          239 non-null    float64
 6   2005          239 non-null    float64
 7   2006          239 non-null    float64
 8   2007          239 non-null    float64
 9   2008          239 non-null    float64
 10  2009          239 non-null    float64
 11  2010          239 non-null    float64
 12  2011          239 non-null    float64
 13  2012          239 non-null    float64
 14  2013          239 non-null    float64
 15  2014          239 non-null    float64
 16  2015          239 non-null    float64
 17  2016          239 non-null    float64
 18  2017          239 non-null    

In [35]:
#Cleaning the dataframe
clean_df = df_2.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 1 to 265
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Name  239 non-null    object 
 1   2000          239 non-null    float64
 2   2001          239 non-null    float64
 3   2002          239 non-null    float64
 4   2003          239 non-null    float64
 5   2004          239 non-null    float64
 6   2005          239 non-null    float64
 7   2006          239 non-null    float64
 8   2007          239 non-null    float64
 9   2008          239 non-null    float64
 10  2009          239 non-null    float64
 11  2010          239 non-null    float64
 12  2011          239 non-null    float64
 13  2012          239 non-null    float64
 14  2013          239 non-null    float64
 15  2014          239 non-null    float64
 16  2015          239 non-null    float64
 17  2016          239 non-null    float64
 18  2017          239 non-null    

In [36]:
#Transforming the dataset to the format Country, Year, CO2
Year_list = years.columns.tolist()
countries = clean_df[clean_df.columns[0]].tolist()
data = pd.DataFrame({'Country': [], 'Year': [], 'CO2': []})
for i in range(0,len(countries)):
    for i_y in range(0,len(Year_list)):
        data_i = pd.DataFrame({'Country': [countries[i]], 'Year': [int(Year_list[i_y])], 
                               'CO2': [clean_df[clean_df['Country Name']==countries[i]].iloc[0][i_y+1]]})
        data = data.append(data_i)
data.head()

,Country,Year,CO2
0,Africa Eastern and Southern,2000.0,0.900399
0,Africa Eastern and Southern,2001.0,0.971981
0,Africa Eastern and Southern,2002.0,0.975749
0,Africa Eastern and Southern,2003.0,1.009963
0,Africa Eastern and Southern,2004.0,1.057182


**1. CO2 emissions by country over the years**

In [37]:
#Making dataframe interactive
idf = data.interactive()

In [38]:
#Define panel widget
year_slider = pn.widgets.IntSlider(name='Year slider', start=2000, end=2018, step=1, value=2000)
year_slider

IntSlider(end=2018, name='Year slider', start=2000, value=2000)

In [39]:
country_button = pn.widgets.Select(name='Select country or region', options=countries)
country_button

Select(options=['Africa Eastern and South...], value='Africa Eastern a...)

In [40]:
co2_pipeline = (
    idf[
        (idf.Country == country_button)
    ]
    .groupby(['Year'])['CO2'].sum()
    .to_frame()
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)

co2_plot = co2_pipeline.hvplot(x='Year',y='CO2',title='CO2 Emissions from 2000 to 2018',
                               ylabel='CO2 Emissions [metric tons per capita]')
co2_plot

**2. Countries with max CO2 emissions level per year**

In [41]:
#Creating the pipeline with the 5 countries with the highest CO2 emissions per year
largest_co2_pipeline = (
    idf[
        (idf.Year == year_slider)
    ]
    .groupby(['Country'])['CO2'].sum().nlargest()
    .to_frame()
    .reset_index()
    .sort_values(by='CO2')
    .reset_index(drop=True)
)
print(type(largest_co2_pipeline))

<class 'hvplot.interactive.Interactive'>


In [45]:
larg_co2_plot = largest_co2_pipeline.hvplot(
    kind = 'bar',
    x = 'Country',
    y = 'CO2',
    title = '5 countries with the highest CO2 emissiones per year'
)

larg_co2_plot

**3. Geomap plot for the CO2 emissions over the years**

In [43]:
#Reading the dataframe with countries available with iso_alpha and iso_num
df_countries= px.data.gapminder()

#Getting a lists of countries available in the dataframe
av_countries = df_countries['country'].unique().tolist()

#Creating a filter to match the countries in the main dataframe
filter1 = data['Country'].isin(av_countries)

#Creating the new dataframe
new_df = data[filter1]

#Indexing the iso code for the available countries

#Getting the exact country
filter2 = new_df['Country'].unique().tolist()

#Computing the iso code for every country
iso_code = pd.DataFrame({'iso_alpha': []})
for i in range(0,len(filter2)):
    for i_c in range (0,len(Year_list)):
        iso_count = df_countries[df_countries['country']==filter2[i]]['iso_alpha'].iloc[0]
        iso_code_i = pd.DataFrame({'iso_alpha': [iso_count]})
        iso_code = iso_code.append(iso_code_i)

#Indexing the iso codes in the dataframe
new_df['iso_alpha'] = iso_code.loc[0,:]

new_df.head(10)

C:\Users\zeama\AppData\Local\Temp\ipykernel_16164\331250328.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Country,Year,CO2,iso_alpha
0,Afghanistan,2000.0,0.037055,AFG
0,Afghanistan,2001.0,0.037488,AFG
0,Afghanistan,2002.0,0.048671,AFG
0,Afghanistan,2003.0,0.057008,AFG
0,Afghanistan,2004.0,0.045700,AFG
0,Afghanistan,2005.0,0.063927,AFG
0,Afghanistan,2006.0,0.073393,AFG
0,Afghanistan,2007.0,0.087083,AFG
0,Afghanistan,2008.0,0.158356,AFG
0,Afghanistan,2009.0,0.211306,AFG


In [44]:
import plotly.express as px
pn.extension('plotly')

fig = px.choropleth(new_df, locations="iso_alpha", color="CO2", hover_name="Country", 
                    animation_frame="Year", projection="natural earth", title='CO2 emissions in the world since 2000 to 2018')

geo_pane = pn.pane.Plotly(fig)
geo_pane

Plotly(Figure)